# Chapter 6 Text Summarization

### Evaluating PEGASUS on the CNN/DailyMail Dataset

In [8]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

summaries = {}

print(f"Features of the dataset: {dataset['train'].column_names}")

Features of the dataset: ['article', 'highlights', 'id']


In [3]:
[len(dataset[split]) for split in dataset.keys()]

[287113, 13368, 11490]

In [4]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/brncat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Summarization baseline

In [9]:
def three_sentence_summary(example):
    return "/n".join(example['article'][:3])